In [1]:
import pandas as pd
from gurobipy import Model, GRB, quicksum
from collections import defaultdict

def solve_vrp_subproblem(tasks_df, agv_df, subproblem_name="Subproblem"):
    """
    단일 AGV 문제 해결을 위한 Gurobi 모델 ('k' 변수 없음)
    """
    if tasks_df.empty:
        return {"solved": False, "routes": {}}
        
    depot = (0, 0)
    points = [(idx + 1, row['x'], row['y']) for idx, row in tasks_df.iterrows()]
    points = [(0, *depot)] + points + [(len(points) + 1, *depot)]

    distance = { i_idx: { j_idx: abs(i_x - j_x) + abs(i_y - j_y) for j_idx, j_x, j_y in points } for i_idx, i_x, i_y in points }

    N = list(range(1, len(points) - 1))
    N_s_t = list(range(0, len(points)))
    
    T_max, v, Q, D = 4, agv_df['speed_cells_per_sec'].iloc[0], agv_df['capacity'].iloc[0], agv_df['max_distance'].iloc[0]
    s, u, deadlines = [0] + tasks_df['service_time'].tolist() + [0], [0] + tasks_df['demand'].tolist() + [0], [0] + tasks_df['deadline'].tolist() + [0]
    
    model = Model(f"AGV_Routing_{subproblem_name}")
    model.setParam('OutputFlag', 0)
    x, y, t = {}, {}, {}

    for r in range(T_max):
        for i in N_s_t:
            for j in N_s_t:
                if i != j: x[i, j, r] = model.addVar(vtype=GRB.BINARY)
        for i in N_s_t:
            y[i, r] = model.addVar(vtype=GRB.BINARY)
            t[i, r] = model.addVar(vtype=GRB.CONTINUOUS)

    late = {i: model.addVar(vtype=GRB.CONTINUOUS) for i in N}
    model.update()

    for r in range(2): model.addConstr(quicksum(x[0, j, r] for j in N) >= 1)
    for r in range(T_max):
        for j in N: model.addConstr(quicksum(x[i, j, r] for i in N_s_t if i != j) == quicksum(x[j, i, r] for i in N_s_t if i != j))
        for j in N_s_t: model.addConstr(quicksum(x[i, j, r] for i in N_s_t if i != j) == y[j, r])
        for i in N_s_t:
            for j in N_s_t:
                if i != j: model.addConstr((x[i, j, r] == 1) >> (t[j, r] >= t[i, r] + distance[i][j] / v + s[i]))
        model.addConstr(quicksum(u[i] * y[i, r] for i in N) <= Q)
        model.addConstr(quicksum(distance[i][j] * x[i, j, r] for i in N_s_t for j in N_s_t if i != j) <= D)
    for j in N: model.addConstr(quicksum(y[j, r] for r in range(T_max)) == 1)
    model.addConstr(t[0, 0] == 0)
    for i in N:
        model.addConstr(late[i] >= 0)
        for r in range(T_max): model.addConstr((y[i, r] == 1) >> (late[i] >= t[i, r] - deadlines[i]))
    end_depot_node = N_s_t[-1]
    for r in range(1, T_max): model.addConstr(t[end_depot_node, r - 1] <= t[0, r])

    model.setObjective(quicksum(late[i] for i in N), GRB.MINIMIZE)
    model.setParam(GRB.Param.TimeLimit, 120)
    model.optimize()

    results = {"solved": False, "routes": {}}
    if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.CUTOFF] and model.SolCount > 0:
        results["solved"] = True
        trips = {}
        for r in range(T_max):
            if sum(y[j, r].X for j in N) > 0.5:
                route, curr_node = [0], 0
                while curr_node != end_depot_node:
                    found_next = False
                    for j_node in N_s_t:
                        if curr_node != j_node and x[curr_node, j_node, r].X > 0.5:
                            route.append(j_node); curr_node = j_node; found_next = True; break
                    if not found_next: break
                if len(route) > 2: trips[r] = route
        results["routes"] = trips
    return results

if __name__ == "__main__":
    # --- 1. 데이터 로드 및 전처리 ---
    try:
        all_tasks = pd.read_csv("./open_v2/task.csv")
        all_agvs = pd.read_csv("./open_v2/agv.csv")
    except FileNotFoundError: exit("Error: 'task.csv' or 'agv.csv' not found.")
    
    # Depot로부터의 거리 계산
    all_tasks['depot_distance'] = abs(all_tasks['x']) + abs(all_tasks['y'])

    # --- 2. 불가능한 Task 사전 제거 ---
    print("--- 1. 처리 불가능한 Task 사전 검증 ---")
    max_fleet_dist = all_agvs['max_distance'].max()
    # 왕복 불가능한 Task 필터링
    unroutable_tasks = all_tasks[all_tasks['depot_distance'] * 2 > max_fleet_dist]
    routable_tasks = all_tasks[all_tasks['depot_distance'] * 2 <= max_fleet_dist]

    if not unroutable_tasks.empty:
        print(f"[경고] {len(unroutable_tasks)}개의 Task는 어떤 AGV로도 왕복이 불가능하여 제외합니다.")
        print(f"  - 제외 Task ID: {unroutable_tasks['task_id'].tolist()}")

    # --- 3. Task 및 AGV 정렬 ---
    print("\n--- 2. Task와 AGV 정렬 ---")
    # Task: Depot에서 먼 순서대로 정렬
    tasks_sorted = routable_tasks.sort_values(by='depot_distance', ascending=False)
    # AGV: 최대 거리 > 속도 > 용량 순서대로 정렬
    agvs_sorted = all_agvs.sort_values(by=['max_distance', 'speed_cells_per_sec', 'capacity'], ascending=False)
    print("Task(먼 곳 우선) 및 AGV(최고 성능 우선) 정렬 완료.")

    # --- 4. 새로운 Greedy 할당 로직 ---
    print("\n--- 3. '가장 먼 Task부터 최고 성능 AGV에게' 할당 시작 ---")
    assignments = defaultdict(list)
    agv_current_demands = defaultdict(float)

    for task_idx, task in tasks_sorted.iterrows():
        assigned = False
        for agv_idx, agv in agvs_sorted.iterrows():
            # 조건 1: AGV가 이 Task를 최소한 왕복할 수 있는가?
            can_reach = task['depot_distance'] * 2 <= agv['max_distance']
            # 조건 2: AGV의 남은 용량이 충분한가?
            has_capacity = agv_current_demands[agv['agv_id']] + task['demand'] <= agv['capacity']
            
            if can_reach and has_capacity:
                assignments[agv['agv_id']].append(task_idx)
                agv_current_demands[agv['agv_id']] += task['demand']
                assigned = True
                print(f"  - Task {task['task_id']} -> AGV {agv['agv_id']} 할당됨")
                break # 다음 Task로
        
        if not assigned:
            print(f"  - [할당 실패] Task {task['task_id']}는 어떤 AGV에도 할당할 수 없습니다 (용량/거리 제약).")
            unroutable_tasks = pd.concat([unroutable_tasks, all_tasks.loc[[task_idx]]])

    # --- 5. 최적화 실행 ---
    print("\n--- 4. 최종 할당에 대한 최적화 실행 ---")
    final_solutions = []
    for agv_id, task_indices in assignments.items():
        if not task_indices: continue
        
        sub_agvs_df = all_agvs[all_agvs['agv_id'] == agv_id]
        sub_tasks_df = all_tasks.loc[task_indices].reset_index(drop=True)
        
        sol = solve_vrp_subproblem(sub_tasks_df, sub_agvs_df, f"Final_AGV_{agv_id}")
        
        if sol['solved'] and sol['routes']:
             sol.update({
                'original_agv_id': agv_id,
                'task_id_map': dict(enumerate(sub_tasks_df['task_id']))
             })
             final_solutions.append(sol)
        else:
            print(f"  - [최적화 실패] AGV {agv_id}가 할당된 Task 그룹의 경로를 찾지 못했습니다.")
            # 최적화 실패 시, 해당 Task들을 최종 unroutable 목록에 추가
            unroutable_tasks = pd.concat([unroutable_tasks, all_tasks.loc[task_indices]])


    # --- 6. 최종 결과 취합 및 출력 ---
    print("\n\n" + "="*60)
    print("           🏁 최종 결합 결과 🏁")
    print("="*60)

    # 최종적으로 처리되지 못한 Task 목록을 한번 더 정리하여 출력
    final_unroutable_ids = unroutable_tasks['task_id'].unique().tolist()
    if final_unroutable_ids:
        print(f"[최종 보고] 총 {len(final_unroutable_ids)}개의 Task가 처리되지 못했습니다.")
        print(f"  - 미처리 Task ID: {final_unroutable_ids}")

    all_agv_full_paths = defaultdict(list)
    for sol in final_solutions:
        agv_id = sol['original_agv_id']
        print(f"\n--- 최종 경로: {agv_id} ---")
        for r, route in sorted(sol["routes"].items()):
            mapped_ids = [sol['task_id_map'][node - 1] for node in route if 0 < node <= len(sol['task_id_map'])]
            print(f"    - Trip {r}: Depot -> {' -> '.join(mapped_ids)} -> Depot")
            all_agv_full_paths[agv_id].extend(["DEPOT"] + mapped_ids + ["DEPOT"])

    if all_agv_full_paths:
        # 최종 결과를 원래 AGV ID 순서로 정렬
        df_data = []
        for agv_id, path in all_agv_full_paths.items():
             clean_path = [path[0]] if path else []
             for node in path[1:]:
                 if node != "DEPOT" or clean_path[-1] != "DEPOT": clean_path.append(node)
             df_data.append({"agv_id": agv_id, "route": ",".join(clean_path)})
        
        final_df = pd.DataFrame(df_data)
        final_df_sorted = final_df.sort_values(by='agv_id').reset_index(drop=True)
        
        output_path = "agv_routes_distance_first_final.csv"
        final_df_sorted.to_csv(output_path, index=False)
        print(f"\n📁 AGV 경로가 CSV 파일로 저장되었습니다: {output_path}")
    else:
        print("\n⚠️ 최종적으로 유효한 경로를 찾지 못했기 때문에 경로를 저장하지 않습니다.")

--- 1. 처리 불가능한 Task 사전 검증 ---

--- 2. Task와 AGV 정렬 ---
Task(먼 곳 우선) 및 AGV(최고 성능 우선) 정렬 완료.

--- 3. '가장 먼 Task부터 최고 성능 AGV에게' 할당 시작 ---
  - Task T0076 -> AGV A015 할당됨
  - Task T0053 -> AGV A001 할당됨
  - Task T0087 -> AGV A001 할당됨
  - Task T0073 -> AGV A002 할당됨
  - Task T0023 -> AGV A005 할당됨
  - Task T0006 -> AGV A001 할당됨
  - Task T0091 -> AGV A003 할당됨
  - Task T0078 -> AGV A015 할당됨
  - Task T0094 -> AGV A003 할당됨
  - Task T0041 -> AGV A006 할당됨
  - Task T0007 -> AGV A005 할당됨
  - Task T0093 -> AGV A003 할당됨
  - Task T0044 -> AGV A004 할당됨
  - Task T0069 -> AGV A004 할당됨
  - Task T0025 -> AGV A014 할당됨
  - Task T0058 -> AGV A014 할당됨
  - Task T0065 -> AGV A014 할당됨
  - Task T0036 -> AGV A006 할당됨
  - Task T0098 -> AGV A012 할당됨
  - Task T0075 -> AGV A012 할당됨
  - Task T0083 -> AGV A012 할당됨
  - Task T0050 -> AGV A007 할당됨
  - Task T0099 -> AGV A007 할당됨
  - Task T0066 -> AGV A007 할당됨
  - Task T0031 -> AGV A013 할당됨
  - Task T0054 -> AGV A013 할당됨
  - Task T0018 -> AGV A010 할당됨
  - Task T0030 -> AGV A010 할

In [2]:
import pandas as pd
from gurobipy import Model, GRB, quicksum
from collections import defaultdict

def solve_vrp_subproblem(tasks_df, agv_df, subproblem_name="Subproblem"):
    """
    단일 AGV 문제 해결을 위한 Gurobi 모델 ('k' 변수 없음)
    """
    if tasks_df.empty:
        return {"solved": False, "routes": {}}
        
    depot = (0, 0)
    points = [(idx + 1, row['x'], row['y']) for idx, row in tasks_df.iterrows()]
    points = [(0, *depot)] + points + [(len(points) + 1, *depot)]

    distance = { i_idx: { j_idx: abs(i_x - j_x) + abs(i_y - j_y) for j_idx, j_x, j_y in points } for i_idx, i_x, i_y in points }

    N = list(range(1, len(points) - 1))
    N_s_t = list(range(0, len(points)))
    
    T_max, v, Q, D = 4, agv_df['speed_cells_per_sec'].iloc[0], agv_df['capacity'].iloc[0], agv_df['max_distance'].iloc[0]
    s, u, deadlines = [0] + tasks_df['service_time'].tolist() + [0], [0] + tasks_df['demand'].tolist() + [0], [0] + tasks_df['deadline'].tolist() + [0]
    
    model = Model(f"AGV_Routing_{subproblem_name}")
    model.setParam('OutputFlag', 0)
    x, y, t = {}, {}, {}

    for r in range(T_max):
        for i in N_s_t:
            for j in N_s_t:
                if i != j: x[i, j, r] = model.addVar(vtype=GRB.BINARY)
        for i in N_s_t:
            y[i, r] = model.addVar(vtype=GRB.BINARY)
            t[i, r] = model.addVar(vtype=GRB.CONTINUOUS)

    late = {i: model.addVar(vtype=GRB.CONTINUOUS) for i in N}
    model.update()

    for r in range(2): model.addConstr(quicksum(x[0, j, r] for j in N) >= 1)
    for r in range(T_max):
        for j in N: model.addConstr(quicksum(x[i, j, r] for i in N_s_t if i != j) == quicksum(x[j, i, r] for i in N_s_t if i != j))
        for j in N_s_t: model.addConstr(quicksum(x[i, j, r] for i in N_s_t if i != j) == y[j, r])
        for i in N_s_t:
            for j in N_s_t:
                if i != j: model.addConstr((x[i, j, r] == 1) >> (t[j, r] >= t[i, r] + distance[i][j] / v + s[i]))
        model.addConstr(quicksum(u[i] * y[i, r] for i in N) <= Q)
        model.addConstr(quicksum(distance[i][j] * x[i, j, r] for i in N_s_t for j in N_s_t if i != j) <= D)
    for j in N: model.addConstr(quicksum(y[j, r] for r in range(T_max)) == 1)
    model.addConstr(t[0, 0] == 0)
    for i in N:
        model.addConstr(late[i] >= 0)
        for r in range(T_max): model.addConstr((y[i, r] == 1) >> (late[i] >= t[i, r] - deadlines[i]))
    end_depot_node = N_s_t[-1]
    for r in range(1, T_max): model.addConstr(t[end_depot_node, r - 1] <= t[0, r])

    model.setObjective(quicksum(late[i] for i in N), GRB.MINIMIZE)
    model.setParam(GRB.Param.TimeLimit, 120)
    model.optimize()

    results = {"solved": False, "routes": {}}
    if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.CUTOFF] and model.SolCount > 0:
        results["solved"] = True
        trips = {}
        for r in range(T_max):
            if sum(y[j, r].X for j in N) > 0.5:
                route, curr_node = [0], 0
                while curr_node != end_depot_node:
                    found_next = False
                    for j_node in N_s_t:
                        if curr_node != j_node and x[curr_node, j_node, r].X > 0.5:
                            route.append(j_node); curr_node = j_node; found_next = True; break
                    if not found_next: break
                if len(route) > 2: trips[r] = route
        results["routes"] = trips
    return results

if __name__ == "__main__":
    # --- 1. 데이터 로드 및 전처리 ---
    try:
        all_tasks = pd.read_csv("./open_v2/task.csv")
        all_agvs = pd.read_csv("./open_v2/agv.csv")
    except FileNotFoundError: exit("Error: 'task.csv' or 'agv.csv' not found.")
    
    # Depot로부터의 거리 계산
    all_tasks['depot_distance'] = abs(all_tasks['x']) + abs(all_tasks['y'])

    # --- 2. 불가능한 Task 사전 제거 ---
    print("--- 1. 처리 불가능한 Task 사전 검증 ---")
    max_fleet_dist = all_agvs['max_distance'].max()
    # 왕복 불가능한 Task 필터링
    unroutable_tasks = all_tasks[all_tasks['depot_distance'] * 2 > max_fleet_dist]
    routable_tasks = all_tasks[all_tasks['depot_distance'] * 2 <= max_fleet_dist]

    if not unroutable_tasks.empty:
        print(f"[경고] {len(unroutable_tasks)}개의 Task는 어떤 AGV로도 왕복이 불가능하여 제외합니다.")
        print(f"  - 제외 Task ID: {unroutable_tasks['task_id'].tolist()}")

    # --- 3. Task 및 AGV 정렬 ---
    print("\n--- 2. Task와 AGV 정렬 ---")
    # Task: Depot에서 먼 순서대로 정렬
    tasks_sorted = routable_tasks.sort_values(by='depot_distance', ascending=False)
    # AGV: 최대 거리 > 속도 > 용량 순서대로 정렬
    agvs_sorted = all_agvs.sort_values(by=['max_distance', 'speed_cells_per_sec', 'capacity'], ascending=False)
    print("Task(먼 곳 우선) 및 AGV(최고 성능 우선) 정렬 완료.")

    # --- 4. 새로운 Greedy 할당 로직 ---
    print("\n--- 3. '가장 먼 Task부터 최고 성능 AGV에게' 할당 시작 ---")
    assignments = defaultdict(list)
    agv_current_demands = defaultdict(float)

    for task_idx, task in tasks_sorted.iterrows():
        assigned = False
        for agv_idx, agv in agvs_sorted.iterrows():
            # 조건 1: AGV가 이 Task를 최소한 왕복할 수 있는가?
            can_reach = task['depot_distance'] * 2 <= agv['max_distance']
            # 조건 2: AGV의 남은 용량이 충분한가?
            has_capacity = agv_current_demands[agv['agv_id']] + task['demand'] <= agv['capacity']
            
            if can_reach and has_capacity:
                assignments[agv['agv_id']].append(task_idx)
                agv_current_demands[agv['agv_id']] += task['demand']
                assigned = True
                print(f"  - Task {task['task_id']} -> AGV {agv['agv_id']} 할당됨")
                break # 다음 Task로
        
        if not assigned:
            print(f"  - [할당 실패] Task {task['task_id']}는 어떤 AGV에도 할당할 수 없습니다 (용량/거리 제약).")
            unroutable_tasks = pd.concat([unroutable_tasks, all_tasks.loc[[task_idx]]])

    # --- 5. 최적화 실행 ---
    print("\n--- 4. 최종 할당에 대한 최적화 실행 ---")
    final_solutions = []
    for agv_id, task_indices in assignments.items():
        if not task_indices: continue
        
        sub_agvs_df = all_agvs[all_agvs['agv_id'] == agv_id]
        sub_tasks_df = all_tasks.loc[task_indices].reset_index(drop=True)
        
        sol = solve_vrp_subproblem(sub_tasks_df, sub_agvs_df, f"Final_AGV_{agv_id}")
        
        if sol['solved'] and sol['routes']:
             sol.update({
                'original_agv_id': agv_id,
                'task_id_map': dict(enumerate(sub_tasks_df['task_id']))
             })
             final_solutions.append(sol)
        else:
            print(f"  - [최적화 실패] AGV {agv_id}가 할당된 Task 그룹의 경로를 찾지 못했습니다.")
            # 최적화 실패 시, 해당 Task들을 최종 unroutable 목록에 추가
            unroutable_tasks = pd.concat([unroutable_tasks, all_tasks.loc[task_indices]])


    # --- 6. 최종 결과 취합 및 출력 ---
    print("\n\n" + "="*60)
    print("           🏁 최종 결합 결과 🏁")
    print("="*60)

    # 최종적으로 처리되지 못한 Task 목록을 한번 더 정리하여 출력
    final_unroutable_ids = unroutable_tasks['task_id'].unique().tolist()
    if final_unroutable_ids:
        print(f"[최종 보고] 총 {len(final_unroutable_ids)}개의 Task가 처리되지 못했습니다.")
        print(f"  - 미처리 Task ID: {final_unroutable_ids}")

    all_agv_full_paths = defaultdict(list)
    for sol in final_solutions:
        agv_id = sol['original_agv_id']
        print(f"\n--- 최종 경로: {agv_id} ---")
        for r, route in sorted(sol["routes"].items()):
            mapped_ids = [sol['task_id_map'][node - 1] for node in route if 0 < node <= len(sol['task_id_map'])]
            print(f"    - Trip {r}: Depot -> {' -> '.join(mapped_ids)} -> Depot")
            all_agv_full_paths[agv_id].extend(["DEPOT"] + mapped_ids + ["DEPOT"])

    if all_agv_full_paths:
        # 최종 결과를 원래 AGV ID 순서로 정렬
        df_data = []
        for agv_id, path in all_agv_full_paths.items():
             clean_path = [path[0]] if path else []
             for node in path[1:]:
                 if node != "DEPOT" or clean_path[-1] != "DEPOT": clean_path.append(node)
             df_data.append({"agv_id": agv_id, "route": ",".join(clean_path)})
        
        final_df = pd.DataFrame(df_data)
        final_df_sorted = final_df.sort_values(by='agv_id').reset_index(drop=True)
        
        output_path = "agv_routes_distance_first_final.csv"
        final_df_sorted.to_csv(output_path, index=False)
        print(f"\n📁 AGV 경로가 CSV 파일로 저장되었습니다: {output_path}")
    else:
        print("\n⚠️ 최종적으로 유효한 경로를 찾지 못했기 때문에 경로를 저장하지 않습니다.")

--- 1. 처리 불가능한 Task 사전 검증 ---

--- 2. Task와 AGV 정렬 ---
Task(먼 곳 우선) 및 AGV(최고 성능 우선) 정렬 완료.

--- 3. '가장 먼 Task부터 최고 성능 AGV에게' 할당 시작 ---
  - Task T0076 -> AGV A015 할당됨
  - Task T0053 -> AGV A001 할당됨
  - Task T0087 -> AGV A001 할당됨
  - Task T0073 -> AGV A002 할당됨
  - Task T0023 -> AGV A005 할당됨
  - Task T0006 -> AGV A001 할당됨
  - Task T0091 -> AGV A003 할당됨
  - Task T0078 -> AGV A015 할당됨
  - Task T0094 -> AGV A003 할당됨
  - Task T0041 -> AGV A006 할당됨
  - Task T0007 -> AGV A005 할당됨
  - Task T0093 -> AGV A003 할당됨
  - Task T0044 -> AGV A004 할당됨
  - Task T0069 -> AGV A004 할당됨
  - Task T0025 -> AGV A014 할당됨
  - Task T0058 -> AGV A014 할당됨
  - Task T0065 -> AGV A014 할당됨
  - Task T0036 -> AGV A006 할당됨
  - Task T0098 -> AGV A012 할당됨
  - Task T0075 -> AGV A012 할당됨
  - Task T0083 -> AGV A012 할당됨
  - Task T0050 -> AGV A007 할당됨
  - Task T0099 -> AGV A007 할당됨
  - Task T0066 -> AGV A007 할당됨
  - Task T0031 -> AGV A013 할당됨
  - Task T0054 -> AGV A013 할당됨
  - Task T0018 -> AGV A010 할당됨
  - Task T0030 -> AGV A010 할

In [3]:
solution2

NameError: name 'solution2' is not defined

In [ ]:
# # -- Part 1 결과 처리 --
# print("\n--- Solution for Part 1 ---")
# if solution1["solved"]:
#     print(f"Objective: {solution1['objective']:.2f}, Travel Cost: {solution1['travel_cost']:.2f}, Tardiness: {solution1['tardiness']:.2f}")
#     for k, trips in solution1["routes"].items():
#         if trips:
#             print(f"  AGV {k}:")
#             for r in sorted(trips.keys()):
#                 route = trips[r]
#                 task_ids = [node - 1 for node in route if node != 0 and node != len(tasks_part1) + 1]
#                 print(f"    Trip {r}: Depot -> {' -> '.join(map(str, task_ids))} -> Depot")
                
#                 # CSV 저장을 위해 경로 포맷팅
#                 formatted_trip = []
#                 for node in route:
#                     if node == 0 or node == len(tasks_part1) + 1:
#                         formatted_trip.append("DEPOT")
#                     else:
#                         task_index = node - 1
#                         formatted_trip.append(f"T{task_index:04d}")
#                 all_agv_full_paths[k].extend(formatted_trip)
# else:
#     print("No solution found for Part 1.")

# # -- Part 2 결과 매핑 및 처리 --
# print("\n--- Solution for Part 2 (with original task IDs) ---")
# if solution2["solved"]:
#     print(f"Objective: {solution2['objective']:.2f}, Travel Cost: {solution2['travel_cost']:.2f}, Tardiness: {solution2['tardiness']:.2f}")
#     for k_local, trips in solution2["routes"].items(): # k_local은 0부터 시작
#         if trips:
#             global_k = k_local + len(agvs2)+  # 전체 AGV 인덱스로 변환
#             print(f"  AGV {global_k}:")
#             for r in sorted(trips.keys()):
#                 route = trips[r]
#                 mapped_task_ids = []
#                 for node in route:
#                     if node != 0 and node != len(tasks_part2) + 1:
#                         # Part 2의 임시 인덱스에 Part 1의 길이를 더해 전체 인덱스로 변환
#                         temp_index = node - 1
#                         original_task_index = temp_index + len(tasks_part1)
#                         mapped_task_ids.append(original_task_index)
#                 print(f"    Trip {r}: Depot -> {' -> '.join(map(str, mapped_task_ids))} -> Depot")

#                 # CSV 저장을 위해 경로 포맷팅
#                 formatted_trip = []
#                 for node in route:
#                     if node == 0 or node == len(tasks_part2) + 1:
#                         formatted_trip.append("DEPOT")
#                     else:
#                         # Part 2의 임시 인덱스에 Part 1의 길이를 더해 전체 인덱스로 변환
#                         temp_index = node - 1
#                         original_task_index = temp_index + len(tasks_part1)
#                         formatted_trip.append(f"T{original_task_index:04d}")
#                 all_agv_full_paths[global_k].extend(formatted_trip) # global_k 사용
# else:
#     print("No solution found for Part 2.")

# # --- 4. 최종 경로 CSV 파일로 저장 ---
# if any(sol["solved"] for sol in [solution1, solution2]):
#     final_csv_data = []
#     for k in sorted(all_agv_full_paths.keys()):
#         full_path = all_agv_full_paths[k]
#         # 중복 DEPOT 제거 (예: ... -> DEPOT, DEPOT -> ...)
#         clean_path = [full_path[0]] if full_path else []
#         for node in full_path[1:]:
#             if node != "DEPOT" or clean_path[-1] != "DEPOT":
#                 clean_path.append(node)
        
#         final_csv_data.append({
#             "agv_id": f"A{k+1:03d}",
#             "route": ",".join(clean_path)
#         })

#     path_df = pd.DataFrame(final_csv_data)
#     output_path = "0930_agv_routes_formatted_1.csv"
#     path_df.to_csv(output_path, index=False)
#     print(f"\n📁 AGV 경로가 CSV 파일로 저장되었습니다: {output_path}")
# else:
#     print("\n⚠️ 최적 해를 찾지 못했기 때문에 경로를 저장하지 않습니다.")

# # -- 전체 결과 요약 --
# total_objective = (solution1['objective'] if solution1['solved'] else 0) + (solution2['objective'] if solution2['solved'] else 0)
# total_cost = (solution1['travel_cost'] if solution1['solved'] else 0) + (solution2['travel_cost'] if solution2['solved'] else 0)
# total_tardiness = (solution1['tardiness'] if solution1['solved'] else 0) + (solution2['tardiness'] if solution2['solved'] else 0)

# print("\n--- Overall Summary ---")
# print(f"Total Estimated Objective: {total_objective:.2f}")
# print(f"Total Travel Cost: {total_cost:.2f}")
# print(f"Total Tardiness: {total_tardiness:.2f}")
# print("="*50)

